# Etape 1 : mise en forme des données

*Durée prévisionnelle : 1h*

Cette première étape nécessite de se confronter aux données réelles qui ne sont pas nécessairement structurées comme souhaitée, et mettre en place les étapes de consolidation (*data management*).

Objectif du notebook :

1. produire un dataset manipulable
2. débuter des statistiques de caractérisation

Choix : ne pas recourir à une base de données car nous sommes dans l'exploratoire. Cela pourrait être une des débouchées de la réflexion.

## 1. Format des données

Les données ont été collectées au fil de l'eau par une interface spécialisée (Gazouilloire, de Sciences Po). Si le tweet fait partie d'un thread, tout le thread est récupéré. La collecte a été lancée à partir du 1ier mars 2020, et dans ce cas ce jeu de données s'arrête en octobre 2020. Il y a un fichier par jour.

Vous pouvez télécharger les données ici : https://www.dropbox.com/scl/fo/21bfj3fq3nac3ix1rccvn/h?rlkey=dk5pxipkfwdx4d709ro294r2i&dl=0

### Comprendre la structuration des données

- Charger un fichier avec pandas ?
- Quelles sont les données ? Qu'est-ce qui caractérise un tweet ?
- Quel est le format ? Quels sont les différents champs ? Que signifient ces champs ?
- Est-ce que tout nous intéresse ? Faire une sélection des champs d'intérêt pour étudier 1/ l'évolution temporelle des tweets ; 2/ les comptes actifs ; 3/ les relations entre les comptes (retweets)

In [1]:
import pandas as pd

On peut spécifier le type de données lors de l'ouverture avec Pandas

In [7]:
rep = "../../../Données/Tweets_HCQ/" # où les données sont téléchargées

df = pd.read_csv(rep + "2020-09-10.csv.gz",
                 dtype = {"id":str,"quoted_id":str,"retweeted_user_id":str}
                )
df.shape

(65587, 57)

In [4]:
df.columns

Index(['id', 'time', 'created_at', 'from_user_name', 'text', 'filter_level',
       'possibly_sensitive', 'withheld_copyright', 'withheld_scope',
       'withheld_countries', 'truncated', 'retweet_count', 'favorite_count',
       'reply_count', 'lang', 'to_user_name', 'to_user_id',
       'in_reply_to_status_id', 'source', 'source_name', 'source_url',
       'location', 'lat', 'lng', 'from_user_id', 'from_user_realname',
       'from_user_verified', 'from_user_description', 'from_user_url',
       'from_user_profile_image_url', 'from_user_utcoffset',
       'from_user_timezone', 'from_user_lang', 'from_user_tweetcount',
       'from_user_followercount', 'from_user_friendcount',
       'from_user_favourites_count', 'from_user_listed',
       'from_user_withheld_scope', 'from_user_withheld_countries',
       'from_user_created_at', 'collected_via_search', 'collected_via_stream',
       'collected_via_thread_only', 'collected_at_timestamp', 'retweeted_id',
       'retweeted_user_name', 'r

On veut garder certains champs

    id
    created_at
    lang
    text
    from_user_name
    from_user_tweetcount
    from_user_followercount
    from_user_friendcount
    retweeted_id
    retweeted_user_name
    quoted_user_id
    mentioned_user_names
    hashtags
    links
    ...


> Penser à travailler sur un sous-échantillon du corpus

## 2. Mise en forme de la base de données

La première étape est de consolider les données

### Ecrire une fonction qui filtre les tweets d'un jour et la tester sur un fichier CSV

- garder uniquement les tweets qui mentionnent le mot clé d'intérêt chloro*
- qui sont en français
- en enlevant les colonnes qui ne nous intéressent pas pour avoir un dataset plus manipulable
- mettre les id en chaînes de caractères pour éviter certains problèmes

In [11]:
nom_fichier = rep + "2020-04-08.csv.gz"

def filtrer_tableau_tweets(nom_fichier):
    """
    Ouvrir et renvoyer le tableau filtré
    """
    tableau = pd.read_csv(nom_fichier,dtype={"id":str,"quoted_user_id":str,
                                             "retweeted_id":str,
                                             "retweeted_user_id":str})
    cols = ["id","created_at","lang","text",
            "from_user_name","from_user_tweetcount","from_user_followercount","from_user_friendcount",
            "retweeted_id","quoted_user_id","mentioned_user_names","hashtags","links",
            'retweeted_user_name', 'retweeted_user_id']
    f_lang = tableau["lang"] == "fr"
    f_mc = tableau["text"].str.lower().str.contains("chloro|chl0r0|chlor0|chl0ro")
    return tableau[f_lang & f_mc][cols]

df = filtrer_tableau_tweets(nom_fichier)

rep_cible = "../../../Données/fichiers_parquet/"

df.to_parquet("2020-04-08.parquet")

/tmp/ipykernel_25234/1239793293.py:7: DtypeWarning: Columns (8,38) have mixed types. Specify dtype option on import or set low_memory=False.
  tableau = pd.read_csv(nom_fichier,dtype={"id":str,"quoted_user_id":str,


### Traiter l'ensemble des données pour construire une structure de données parquet qui ne contient que les données filtrées

- Convertir un fichier csv en parquet avec pandas
- Tester la vitesse d'ouverture
- Convertir l'ensemble des fichiers en chargeant les CSV, en les filtrant, en conservant les éléments filtrés dans une liste, puis en écrivant l'ensemble dans un fichier parquet

Test le la lecture des fichiers parquets

In [32]:
df.to_parquet("test.parquet")
df.to_csv("test.csv.gz")

In [36]:
%time df = pd.read_parquet("test.parquet")
%time df = pd.read_csv("test.csv.gz")

CPU times: user 67.3 ms, sys: 24 ms, total: 91.3 ms
Wall time: 64 ms
CPU times: user 246 ms, sys: 12.1 ms, total: 258 ms
Wall time: 257 ms


> Pour faciliter la gestion mémoire : lire chaque csv en chunk et sauvegarder chaque fichier dans un format parquet dans un dossier commun. Mais attention, il y a un enjeu à bien définir un format commun sinon il peut y avoir des soucis.

Sur Parquet & CSV https://www.icem7.fr/cartographie/parquet-devrait-remplacer-le-format-csv/

> Il est possible de créer un seul fichier parquet, mais pour cela il faut gérer correctement le schéma de la base avec `pyarrow`

In [ ]:
import os

# Dossiers concernés
rep = "../../../Données/Tweets_HCQ/"

# ne garder que les noms de fichiers qui ont "2020" dedans (correspondant à notre période)
fichiers = [i for i in os.listdir(rep) if "2020" in i]

# liste des tweets filtrés
corpus = []

# Boucle sur tous les fichiers
for f in fichiers:
    # print(repertoire + f)
    # lire et filtrer un fichier avec la fonction
    df = filtrer_tableau_tweets(rep + f)
    
    # ajouter au corpus
    corpus.append(df)

# concaténer l'ensemble des tweets dans un dataframe
df = pd.concat(corpus)

# Écrire dans un fichier parquet unique
df.to_parquet("dataset.parquet")

### Charger les données filtrées, supprimer les doublons et compléter la base en calculant le nombre de reweets

Charger les données (soit le dossier ; soit le fichier)

In [14]:
df = pd.read_parquet("dataset.parquet")
df.shape

(3788348, 15)

Dédoublonnons

In [15]:
df = df.drop_duplicates(subset=["id"])

Calculer les retweets et les ajouter au dataset

In [16]:
retweets = df["retweeted_id"].value_counts()
df = df.join(retweets,on="id")
df = df.rename(columns={"count":"retweet"})

Sauvegarder le nouveau dataset

In [17]:
df.to_parquet("dataset.parquet")

## 3. Faire des statistiques sur les tweets (comptage)

### Débuter l'exploration des données avec des statistiques descriptives du corpus

- nombre de tweets originaux
- nombre de retweet
- comptes ayant le plus de tweets originaux
    - regarder les tweets associés
- Identifier les tweets les plus retweetés du corpus
- comptes ayant le plus de retweets total (plus grande viralité)

Proportion tweets originaux / total des tweets

In [18]:
pd.isnull(df["retweeted_id"]).sum()/len(df)

0.17182932388592123

Calculer le nombre de tweets originaux par utilisateur

In [80]:
df["original"] = pd.isnull(df["retweeted_id"])
df.groupby("from_user_name")["original"].sum().sort_values()

from_user_name
00000CHANEL          0
audescande           0
auderaiplie          0
audemazoue           0
audemannoury         0
                  ... 
lan1794           1333
InfoVeryfiable    1337
GBOU66            1824
zenutopia1        2022
a_1_0_2           2038
Name: original, Length: 442388, dtype: int64

Nombre total de retweets obtenus par les comptes

In [82]:
df.groupby("from_user_name")["retweet"].sum().sort_values()

from_user_name
00000CHANEL             0.0
chicaa135               0.0
chibron_bac2bac         0.0
chibre59                0.0
chibdan4christ          0.0
                     ...   
CorinneReverbel     33655.0
Conflits_FR         41617.0
medicalfollower     77274.0
biobiobiobior      136669.0
raoult_didier      189286.0
Name: retweet, Length: 442388, dtype: float64

### (pas traité en cours) Aller vers une visualisation

- Visualiser la distribution des retweets par tweet
- Proposer une visualisation de la diversité de l'activité des comptes (volume de tweets, viralité, taille des comptes)

> Pour la visualisation : 
>- en x : nombre d'abonnés
>- en y : nombre de tweets fait
>- en taille nombre de retweet

### (pas traité en cours) Bonus : Analyser les biographies des comptes

- Sortir un fichier comptes/bio pour les comptes qui ont fait au moins 10 tweets originaux
    - Quelle bio sélectionner ?

Construire un jeu de données par comptes qui ont au moins 2 activités :

- nom
- bio
- date premier tweet/retweet
- date dernier tweet/retweet
- nombre de tweet
- nombre de retweet